In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Using device: cuda:0


In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
seed = 25
# random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
train_data = pd.read_csv('datasets/subtask_2/es/train.tsv',sep='\t')
train_data = train_data.reset_index(drop=True)
print(train_data.head())

      id                                               text label
0  12786  Sin embargo, los jóvenes son capaces de recono...     B
1  12361  ¿Hay algo más que quieras compartir? ¿Algo sob...     B
2   1662  El servicio de sala es bueno, rápido y amabilí...     B
3  14729  Para concentrarse en el hablante, trata de des...     F
4   9312  Los responsables locales tendrán ahora que esp...     F


In [5]:
train_data_texts = train_data['text'].to_list()
train_data_labels = train_data['label'].to_list()
print("Unique labels: ", set(train_data_labels))

Unique labels:  {'C', 'F', 'A', 'E', 'D', 'B'}


In [6]:
from sklearn.model_selection import train_test_split
train_data_texts = train_data['text'].to_list()
train_data_labels = train_data['label'].to_list()
train_texts, test_texts, train_labels, test_labels = train_test_split(train_data_texts, train_data_labels, test_size=0.1, random_state=25)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.1, random_state=25)
print('train data size: ', len(train_texts))
print('validation data size: ', len(val_texts))
print('test data size: ', len(test_texts))

train data size:  17766
validation data size:  1975
test data size:  2194


In [7]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", num_labels=6).to(device)
print("Model Configurations")
print()
print(model.config)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Model Configurations

BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}



In [8]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=256)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels[idx] == 'A':
            item['labels'] = torch.tensor(0)
        elif self.labels[idx] == 'B':
            item['labels'] = torch.tensor(1)
        elif self.labels[idx] == 'C':
            item['labels'] = torch.tensor(2)
        elif self.labels[idx] == 'D':
            item['labels'] = torch.tensor(3)
        elif self.labels[idx] == 'E':
            item['labels'] = torch.tensor(4)
        elif self.labels[idx] == 'F':
            item['labels'] = torch.tensor(5)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

print("Sample Data Point")
print()
print(train_dataset[0])

Sample Data Point

{'input_ids': tensor([    4,  1094,  1135,  1162,  2655,  1094,  1072,  1735,  1094,  3828,
        21178,  1036,  1065,  2847,  5218,  1181,     3,  2346,  1094,  1009,
         1147,  1094,  1135,  1162,  4425,  1008,  3842,  1943,  1065,  2120,
         1008, 15061, 11262,  7988, 27347,  2218,  1040,  4425, 12112,  6737,
         1008,  3842, 11694,  1036,  1065,  5218,  1008,  1365,  2193,  1017,
         1038,  1062,  7298, 30957,  1096,  1040,  3673,  1694,  1009,  1115,
         1135,  1264, 21392,  1040,  1735,  1115,  1036,  1040,  3385,  5087,
         1009,  1244,  1135,  1264, 21392,  1040,  1735,  1244,  1036,  1040,
         3385,  2812,  1009,  1338,  1135,  1162,  2655,  1094,  1072,  6075,
            5,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,   

In [9]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [10]:
training_args = TrainingArguments(
    output_dir='./models/spanish_bert_task2',       
    num_train_epochs=8,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/home2/harshit.g/miniconda3/envs/pyt/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17766
  Num Epochs = 8
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 1
  Total optimization steps = 1488
  Number of trainable parameters = 109855494
/home2/harshit.g/miniconda3/envs/pyt/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input t

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.109600,1.116789,0.487595,0.453349,0.486235,0.487595
2,0.914100,0.924748,0.571646,0.539851,0.572042,0.571646
3,0.832000,0.901094,0.585823,0.561209,0.615051,0.585823
4,0.636200,0.915712,0.607089,0.601434,0.603577,0.607089
5,0.463100,1.090115,0.601519,0.594738,0.601158,0.601519
6,0.257600,1.376799,0.597975,0.594619,0.616179,0.597975
7,0.119700,1.599416,0.602025,0.603086,0.633673,0.602025
8,0.063800,1.736719,0.602532,0.596942,0.630355,0.602532


***** Running Evaluation *****
  Num examples = 1975
  Batch size = 96
Saving model checkpoint to ./models/spanish_bert_task2/checkpoint-186
Configuration saved in ./models/spanish_bert_task2/checkpoint-186/config.json
Model weights saved in ./models/spanish_bert_task2/checkpoint-186/pytorch_model.bin
/home2/harshit.g/miniconda3/envs/pyt/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 1975
  Batch size = 96
Saving model checkpoint to ./models/spanish_bert_task2/checkpoint-372
Configuration saved in ./models/spanish_bert_task2/checkpoint-372/config.json
Model weights saved in ./models/spanish_bert_task2/checkpoint-372/pytorch_model.bin
/home2/harshit.g/miniconda3/envs/pyt/lib/python3.10/site-packages/torch/nn/parallel/_functions

TrainOutput(global_step=1488, training_loss=0.6148301084595983, metrics={'train_runtime': 1383.2581, 'train_samples_per_second': 102.749, 'train_steps_per_second': 1.076, 'total_flos': 1.6215014895682176e+16, 'train_loss': 0.6148301084595983, 'epoch': 8.0})

In [11]:
trainer.evaluate(test_dataset)
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

***** Running Evaluation *****
  Num examples = 2194
  Batch size = 96
/home2/harshit.g/miniconda3/envs/pyt/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


***** Running Prediction *****
  Num examples = 2194
  Batch size = 96
/home2/harshit.g/miniconda3/envs/pyt/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


              precision    recall  f1-score   support

           0       0.69      0.70      0.69       317
           1       0.44      0.47      0.45       354
           2       0.49      0.46      0.48       370
           3       0.65      0.49      0.56       378
           4       0.49      0.53      0.51       369
           5       0.80      0.91      0.85       406

    accuracy                           0.60      2194
   macro avg       0.59      0.59      0.59      2194
weighted avg       0.60      0.60      0.59      2194



In [12]:
trainer.save_model('./models/spanish_bert_task2/trained_model')

Saving model checkpoint to ./models/spanish_bert_task2/trained_model
Configuration saved in ./models/spanish_bert_task2/trained_model/config.json
Model weights saved in ./models/spanish_bert_task2/trained_model/pytorch_model.bin
